In [1]:
import pandas as pd
import numpy as np
import datetime
import joblib

from tensorflow.keras.models import load_model


In [ ]:
MODEL_PATH = "mvp_lstm_model.h5"
SCALER_PATH = "mvp_scaler.pkl"
PREDICTION_CSV = "final_predictions.csv"
LOOK_BACK = 60
PREDICTION_HORIZONS = [1, 7, 30]  

print("Loading model and scalers...")
model = load_model(MODEL_PATH)
scaler_dict = joblib.load(SCALER_PATH)

Loading model and scalers...


In [ ]:
CSV_FILE = "crypto_price_data.csv"
df = pd.read_csv(CSV_FILE)
df.sort_values(by=["Ticker", "Date"], inplace=True)
df.reset_index(drop=True, inplace=True)


In [ ]:
frames = []
for ticker in df["Ticker"].unique():
    sub = df[df["Ticker"] == ticker].copy()
    if len(sub) < LOOK_BACK:
        print(f"Skipping {ticker}: Not enough data for a 60-day sequence.")
        continue
    
    # Grab last 60 rows
    sub_60 = sub.iloc[-LOOK_BACK:].copy()
    frames.append(sub_60)

if not frames:
    raise ValueError("No ticker had 60 days of data to predict on.")

df_last_60 = pd.concat(frames).reset_index(drop=True)


In [5]:
# --- 4. Scale Data & Create a 60-day Sequence for Each Ticker ---
predictions = []
timestamp_utc = datetime.datetime.now(datetime.timezone.utc).replace(microsecond=0).isoformat()

for ticker in df_last_60["Ticker"].unique():
    sub = df_last_60[df_last_60["Ticker"] == ticker].copy()
    
    # Ensure we have a scaler for this ticker
    if ticker not in scaler_dict:
        print(f"Warning: No scaler found for {ticker}. Skipping.")
        continue
    
    scaler = scaler_dict[ticker]
    
    # Scale "Close" column
    sub["CloseScaled"] = scaler.transform(sub[["Close"]])
    
    # Prepare X_input from the entire 60-day window
    X_input = sub["CloseScaled"].values.reshape(1, LOOK_BACK, 1)  # shape => (1, 60, 1)
    
    # Predict (EOD, EOW, EOM) in scaled domain
    pred_scaled = model.predict(X_input)[0]  # shape => (3,)
    
    # Inverse transform predictions
    pred_real = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
    
    # Store results
    pred_dict = {
        "timestamp_utc": timestamp_utc,
        "ticker": ticker,
        "pred_eod": pred_real[0],
        "pred_eow": pred_real[1],
        "pred_eom": pred_real[2],
    }
    predictions.append(pred_dict)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━

In [6]:
# --- 5. Save Predictions to CSV ---
pred_df = pd.DataFrame(predictions)
pred_df.to_csv(PREDICTION_CSV, index=False)

print(f"Final predictions saved to {PREDICTION_CSV}")

Final predictions saved to final_predictions.csv
